<a href="https://colab.research.google.com/github/tsomerville2/drtrain/blob/main/speedLlama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/turboderp/exllama
%cd exllama
!pip install -r requirements.txt

Cloning into 'exllama'...
remote: Enumerating objects: 1316, done.
remote: Counting objects: 100% (652/652), done.
remote: Compressing objects: 100% (200/200), done.
remote: Total 1316 (delta 533), reused 474 (delta 452), pack-reused 664
Receiving objects: 100% (1316/1316), 899.07 KiB | 7.49 MiB/s, done.
Resolving deltas: 100% (921/921), done.
/content/exllama
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 16.9 MB/s eta 0:00:00


In [ ]:
import os
# Make sure you have git-lfs installed (https://git-lfs.com)
!git lfs install

# clone without large files – just their pointers
os.environ['GIT_LFS_SKIP_SMUDGE'] = '1'  # replace 'VAR_NAME' and 'value' with your values
!git clone https://huggingface.co/TheBloke/Wizard-Vicuna-13B-Uncensored-GPTQ

Updated git hooks.
Git LFS initialized.
Cloning into 'Wizard-Vicuna-13B-Uncensored-GPTQ'...
remote: Enumerating objects: 57, done.
remote: Total 57 (delta 0), reused 0 (delta 0), pack-reused 57
Unpacking objects: 100% (57/57), 494.23 KiB | 5.68 MiB/s, done.


In [ ]:
import requests
from tqdm.auto import tqdm

%cd Wizard-Vicuna-13B-Uncensored-GPTQ
# URLs
tensors_url = 'https://huggingface.co/TheBloke/Wizard-Vicuna-13B-Uncensored-GPTQ/resolve/main/Wizard-Vicuna-13B-Uncensored-GPTQ-4bit-128g.compat.no-act-order.safetensors'
tokens_url = 'https://huggingface.co/TheBloke/Wizard-Vicuna-13B-Uncensored-GPTQ/resolve/main/tokenizer.model'

# Function to download with progressbar
def download_with_progressbar(url, filename):
    response = requests.get(url, stream=True)
    response.raise_for_status()  # Ensure we got a valid response

    total_size_in_bytes= int(response.headers.get('content-length', 0))
    block_size = 1024 #1 Kbyte

    progress_bar = tqdm(total=total_size_in_bytes, unit='iB', unit_scale=True)

    with open(filename, 'wb') as file:
        for data in response.iter_content(block_size):
            progress_bar.update(len(data))
            file.write(data)
    progress_bar.close()
    if total_size_in_bytes != 0 and progress_bar.n != total_size_in_bytes:
        print("ERROR, something went wrong")

# Download files
print("Downloading safetensors...")
download_with_progressbar(tensors_url, 'Wizard-Vicuna-13B-Uncensored-GPTQ-4bit-128g.compat.no-act-order.safetensors')

print("Downloading tokenizer model...")
download_with_progressbar(tokens_url, 'tokenizer.model')
%cd ..

/content/exllama/Wizard-Vicuna-13B-Uncensored-GPTQ


  0%|          | 0.00/8.11G [00:00<?, ?iB/s]

  0%|          | 0.00/500k [00:00<?, ?iB/s]

/content/exllama


In [ ]:
from model import ExLlama, ExLlamaCache, ExLlamaConfig
from tokenizer import ExLlamaTokenizer
from generator import ExLlamaGenerator
import os, glob

# Directory containing model, tokenizer, generator

model_directory =  "./Wizard-Vicuna-13B-Uncensored-GPTQ"

# Locate files we need within that directory

tokenizer_path = os.path.join(model_directory, "tokenizer.model")
model_config_path = os.path.join(model_directory, "config.json")
st_pattern = os.path.join(model_directory, "*.safetensors")
model_path = glob.glob(st_pattern)[0]

# Create config, model, tokenizer and generator

config = ExLlamaConfig(model_config_path)               # create config from config.json
config.model_path = model_path                          # supply path to model weights file

model = ExLlama(config)                                 # create ExLlama instance and load the weights
tokenizer = ExLlamaTokenizer(tokenizer_path)            # create tokenizer from tokenizer model file

cache = ExLlamaCache(model)                             # create cache for inference
generator = ExLlamaGenerator(model, tokenizer, cache)   # create generator

# Configure generator

generator.disallow_tokens([tokenizer.eos_token_id])

generator.settings.token_repetition_penalty_max = 1.2
generator.settings.temperature = 0.95
generator.settings.top_p = 0.65
generator.settings.top_k = 100
generator.settings.typical = 0.5

In [ ]:
# Produce a simple generation

prompt = "Hello world,"
print (prompt, end = "")

output = generator.generate_simple(prompt, max_new_tokens = 200)

print(output[len(prompt):])

Hello world, this is my first post.
15. 
I'm a beginner in programming and I hope to learn more from you guys! Thank you for reading my post. Bye bye :)

Here are some code snippets that demonstrate how to use the Python `random` module:
```python
import random
# generate a list of random numbers between 0-9
numbers = [random.randint(0, 9) for _ in range(10)]
print("Random Numbers:", numbers)
# shuffle the list randomly
shuffled_list = random.sample(range(len(numbers)), len(numbers))
print("Shuffled List:", shuffled_list)
# choose a random number from the list
chosen_number = random.choice(numbers)
print("Ch


In [ ]:
import json

# Open the file
with open('logs.txt', 'r') as f:
    # Read the lines
    lines = f.readlines()

# Initialize an empty array to store the data
data = []
fulltext = ""
# Loop over the lines
for line in lines:
    fulltext += line
for chunk in fulltext.split("----"):
    # Split the line by the delimiter and add to the array
    data.append(chunk)

# Now the 'data' array contains the contents of the text file, split by the delimiter

# Initialize a list to store the logs
logs = []

# Loop over the data
for chunk in data:
    print(input)
    # Generate the output from the model
    output = generator.generate_simple(chunk, max_new_tokens=200)
    print(output)
    # Append the input and output to the logs
    logs.append({
        'input': chunk,
        'output': output[len(chunk):]
    })

# Write the logs to a JSON file
with open('logs.json', 'w') as f:
    json.dump(logs, f)


<bound method Kernel.raw_input of <google.colab._kernel.Kernel object at 0x7c2ea7772140>>
Entry 2/6/23

Today was a difficult day. I felt overwhelmed and like I couldn't handle anything. I was able to get out of bed, but I struggled to focus on anything. I felt like my thoughts were in a fog and I just wanted to crawl back into bed and sleep. I'm trying my best to stay positive and remember that this won't last forever, but it's hard.
What coping strategies do you find helpful when you're feeling overwhelmed?


Given the above, extract a few key notes relevant to a therapist, and use dashes for each note:
- Client feels overwhelmed often
- Struggles with focusing and concentration
- Feels like their thoughts are in a fog
- Wants to retreat to safety of bed
- Coping strategies sought after (e.g., mindfulness meditation)
- Efforts to maintain positivity despite difficulty

Note any other significant information or patterns that emerge from the journal entries as well. This could include 

In [ ]:
#run a chatbot
!python example_chatbot.py -d ./Wizard-Vicuna-13B-Uncensored-GPTQ -un "Malcolm" -p prompt_chatbort.txt

In [ ]:
#benchmark inference speed
!python test_benchmark_inference.py -d ./Wizard-Vicuna-13B-Uncensored-GPTQ -p -ppl